# Compare SQLs

# Dataset

In [1]:
import pandas as pd
# load csv
db_train = pd.read_csv('llm/data/txt2sql_alerce_train_v4_0.csv')
db_test = pd.read_csv('llm/data/txt2sql_alerce_test_v4_0.csv')

In [2]:
import os
import sqlalchemy as sa
import requests
import time
# Create a connection to the ALeRCE database
def create_conn_alerce(access_time: int = 2):
  """
  Create a connection to the ALeRCE database.
  
  Args:
    access_time: Integer representing the time limit for the connection.
    2 for default access, 10 for extended access.
    
  Returns:
    SQLAlchemy engine object
  
  Raises:
    ValueError: If URL fetch fails or credentials are invalid
  """
  # Common URL for both access levels
  url = "https://raw.githubusercontent.com/alercebroker/usecases/master/alercereaduser_v4.json"
  
  # For security, get extended access credentials from environment variables
  user_extended = os.environ.get('ALERCE_USER_EXTENDED', '')
  pass_extended = os.environ.get('ALERCE_PASS_EXTENDED', '')
  
  n_tries = 3
  params = None
  
  # Fetch parameters from URL with retry logic
  for n_try in range(n_tries):
    try:
      response = requests.get(url)
      if response.status_code != 200:
        if n_try < n_tries - 1:
          time.sleep(2 ** n_try)  # exponential backoff
          continue
        else:
          raise ValueError(f"Failed to fetch URL: {url}, Status Code: {response.status_code}")
      
      params = response.json().get('params')
      if not params:
        raise ValueError("Missing 'params' in the JSON response")
      break
        
    except requests.RequestException as e:
      if n_try < n_tries - 1:
        time.sleep(2 ** n_try)
        continue
      else:
        raise ValueError(f"Network error when fetching {url}: {str(e)}")
    except ValueError as e:
      if "JSON" in str(e):
        raise ValueError("Invalid JSON response from URL")
      else:
        raise e
  
  # Create connection string based on access level
  if access_time == 2:
    conn_string = f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}/{params['dbname']}"
  elif access_time == 10:
    if not user_extended or not pass_extended:
      raise ValueError("Extended access credentials not found in environment variables")
    conn_string = f"postgresql+psycopg2://{user_extended}:{pass_extended}@{params['host']}/{params['dbname']}"
  else:
    raise ValueError(f'Access time {access_time} not supported')
  
  # Create and return engine with connection pooling disabled
  engine = sa.create_engine(conn_string, poolclass=sa.pool.NullPool)
  return engine


def run_sql_alerce(
    sql: str, 
    access_time: int = 2, 
    n_tries: int = 3, 
    query_time: bool = False
):
  ''' Execute the SQL query at the ALeRCE database and return the result
    Args:
        sql: SQL query to execute
        access_time: Integer representing the time limit for the connection. 
        2 for default access, 10 for extended access.
        n_tries: Number of tries to execute the query
        query_time: Boolean indicating whether to track query execution time
    Returns:
    query: pandas dataframe with the result of the query
    error: error message if the query could not be executed
    execution_time: time taken to execute the query    
  '''

  try:
    engine = create_conn_alerce(access_time=access_time)
  except ValueError as e:
    return None, e
    
  query = None
  error = None
  execution_time = None
  
  try:
    for n_try in range(n_tries):
      try:
        with engine.begin() as conn:
          start_time = time.time()
          query = pd.read_sql_query(sa.text(sql), conn)
          if query_time:
            execution_time = time.time() - start_time
          error = None
          break
      except Exception as e:
        error = e
        if n_try < n_tries - 1:
          time.sleep(2 ** n_try)  # exponential backoff
        else:
          # Last attempt failed, keep the error
          pass
  
  finally:
    # Always dispose of the engine to close connections
    engine.dispose()

  if query_time:
    return query, error, execution_time
  else:
    return query, error




## Running test of train and test queries


In [3]:
gold_columns_train = []
# iter db_train 
n = 5
for indx, row in db_train.iterrows():
    
    query = row['gold_query']
    tables_i, error = run_sql_alerce( query, access_time=2, n_tries=n, query_time=False )
    if error:
        print(f'Error in query {row.req_id}: {error}')
        gold_columns_train.append({"req_id": row.req_id, "cols": [], "shape": []})
    else:
        print(f'Query {row.req_id} executed successfully, shape: {tables_i.shape}')
        gold_columns_train.append({"req_id": row.req_id, "cols": tables_i.columns.values.tolist(), "shape": tables_i.shape})

Query 13 executed successfully, shape: (4471, 4)
Query 10 executed successfully, shape: (184, 3)
Query 15 executed successfully, shape: (42, 7)
Query 4 executed successfully, shape: (1537, 8)
Query 25 executed successfully, shape: (4, 30)
Query 7 executed successfully, shape: (896, 2)
Query 2 executed successfully, shape: (5732, 2)
Query 3 executed successfully, shape: (11, 6)
Query 23 executed successfully, shape: (668355, 28)
Query 32 executed successfully, shape: (2, 5)
Query 20 executed successfully, shape: (41262, 28)
Query 18 executed successfully, shape: (1041, 11)
Query 35 executed successfully, shape: (10, 6)
Query 11 executed successfully, shape: (491, 4)
Query 9 executed successfully, shape: (54, 8)
Query 5 executed successfully, shape: (781, 9)
Query 0 executed successfully, shape: (8218, 6)
Query 28 executed successfully, shape: (34, 1)
Query 36 executed successfully, shape: (178, 3)
Query 38 executed successfully, shape: (1247, 22)
Query 24 executed successfully, shape: (

In [4]:
gold_columns_test = []
# iter db_test 
n = 5
for indx, row in db_test.iterrows():
    
    query = row['gold_query']
    tables_i, error = run_sql_alerce( query, access_time=2, n_tries=n, query_time=False )
    if error:
        print(f'Error in query {row.req_id}: {error}')
        gold_columns_test.append({"req_id": row.req_id, "cols": [], "shape": []})
    else:
        print(f'Query {row.req_id} executed successfully, shape: {tables_i.shape}')
        gold_columns_test.append({"req_id": row.req_id, "cols": tables_i.columns.values.tolist(), "shape": tables_i.shape})

Query 8 executed successfully, shape: (186, 28)
Query 27 executed successfully, shape: (5520, 11)
Error in query 17: (psycopg2.errors.QueryCanceled) canceling statement due to statement timeout

[SQL: 
SELECT
  *
FROM
  (
SELECT *
FROM (
SELECT
  feature.oid, feature.value, feature.name, feature.version, feature.fid
FROM
  (
SELECT
    object.oid, object.meanra, object.meandec
FROM
    object INNER JOIN
    probability
    ON object.oid = probability.oid
WHERE
    probability.classifier_name='lc_classifier'
    AND probability.class_name = 'AGN'
    AND probability.ranking = 1
) as obj_oids
    INNER JOIN
    feature ON feature.oid = obj_oids.oid
WHERE
  feature.name = 'ExcessVar'
  AND feature.fid = 1
  AND feature.value < -0.001
ORDER BY feature.value ASC
) as sq1
UNION
SELECT *
FROM (
SELECT
  feature.oid, feature.value, feature.name, feature.version, feature.fid
FROM
  (
SELECT
    object.oid, object.meanra, object.meandec
FROM
    object INNER JOIN
    probability
    ON object.oi

## Columns

In [5]:
gold_columns_train

[{'req_id': 13,
  'cols': ['oid', 'class_name', 'lastmjd', 'firstmjd'],
  'shape': (4471, 4)},
 {'req_id': 10, 'cols': ['oid', 'sn_prob', 'cv_prob'], 'shape': (184, 3)},
 {'req_id': 15,
  'cols': ['oid',
   'probability',
   'candid',
   'fid',
   'mjd',
   'magstat_fid',
   'dmdt_first'],
  'shape': (42, 7)},
 {'req_id': 4,
  'cols': ['oid',
   'fid',
   'dmdt_first',
   'candid',
   'f_id',
   'magpsf',
   'sigmapsf_corr',
   'sigmapsf_corr_ext'],
  'shape': (1537, 8)},
 {'req_id': 25,
  'cols': ['oid',
   'ndethist',
   'ncovhist',
   'mjdstarthist',
   'mjdendhist',
   'corrected',
   'stellar',
   'ndet',
   'g_r_max',
   'g_r_max_corr',
   'g_r_mean',
   'g_r_mean_corr',
   'meanra',
   'meandec',
   'sigmara',
   'sigmadec',
   'deltajd',
   'firstmjd',
   'lastmjd',
   'step_id_corr',
   'diffpos',
   'reference_change',
   'oid',
   'classifier_name',
   'class_name',
   'probability',
   'ranking',
   'source_id',
   'ra',
   'dec'],
  'shape': (4, 30)},
 {'req_id': 7, 'cols'

In [6]:
gold_columns_test


[{'req_id': 8,
  'cols': ['oid',
   'ndethist',
   'ncovhist',
   'mjdstarthist',
   'mjdendhist',
   'corrected',
   'stellar',
   'ndet',
   'g_r_max',
   'g_r_max_corr',
   'g_r_mean',
   'g_r_mean_corr',
   'meanra',
   'meandec',
   'sigmara',
   'sigmadec',
   'deltajd',
   'firstmjd',
   'lastmjd',
   'step_id_corr',
   'diffpos',
   'reference_change',
   'oid',
   'classifier_name',
   'classifier_version',
   'class_name',
   'probability',
   'ranking'],
  'shape': (186, 28)},
 {'req_id': 27,
  'cols': ['oid',
   'meanra',
   'meandec',
   'ndet',
   'firstmjd',
   'deltajd',
   'g_r_max',
   'classifier_name',
   'class_name',
   'ranking',
   'probability'],
  'shape': (5520, 11)},
 {'req_id': 17, 'cols': [], 'shape': []},
 {'req_id': 37,
  'cols': ['oid',
   'classifier_name',
   'classifier_version',
   'class_name',
   'probability',
   'ranking'],
  'shape': (200, 6)},
 {'req_id': 34,
  'cols': ['oid', 'name', 'value', 'fid', 'version'],
  'shape': (354, 5)},
 {'req_id

## Test single query

In [ ]:

query = '''
SELECT
    ss_ztf.*, detection.mjd, detection.fid,
    detection.isdiffpos, detection.ra, detection.dec,
    detection.magpsf, detection.sigmapsf
FROM
    ss_ztf INNER JOIN
    detection
    ON ss_ztf.oid = detection.oid
    AND ss_ztf.candid = detection.candid
WHERE
    ssnamenr IN ('2003FP134','2009UK56')
    AND ssdistnr < 2
ORDER BY ssnamenr, mjd
'''
query_columns = []
n = 3
tables_i, error = run_sql_alerce( query, access_time=2, n_tries=n, query_time=False )
if error:
    print(f'Error in query: {error}')
    # query_columns.append({"req_id": row.req_id, "cols": [], "shape": []})
else:
    print(f'Query executed successfully, shape: {tables_i.shape}')
    # query_columns.append({"req_id": row.req_id, "cols": tables_i.columns.values.tolist(), "shape": tables_i.shape})
tables_i

Query executed successfully, shape: (27, 12)


,oid,candid,ssdistnr,ssmagnr,ssnamenr,mjd,fid,isdiffpos,ra,dec,magpsf,sigmapsf
0,ZTF21abbsmyp,1595376262715015044,0.0,19.5,2003FP134,59349.376262,1,1,256.824875,-25.974222,20.192154,0.178280
1,ZTF21abcqekm,1608313551615015025,0.0,19.0,2003FP134,59362.313553,2,1,254.248201,-24.971674,18.976067,0.170550
2,ZTF21abculin,1609376562915015021,0.0,19.0,2003FP134,59363.376562,2,1,254.000931,-24.876152,19.098774,0.178708
3,ZTF21abdihfx,1612381041715015014,0.0,18.9,2003FP134,59366.381042,2,1,253.289819,-24.597101,19.052930,0.091880
4,ZTF21abdulcm,1614399211715015002,0.0,18.8,2003FP134,59368.399213,2,1,252.804054,-24.403010,18.866552,0.111349
5,ZTF21abeywpa,1619365033915015076,0.0,19.1,2003FP134,59373.365035,2,1,251.609834,-23.908624,19.000805,0.124175
6,ZTF21abfkygj,1621376883815015021,0.0,19.2,2003FP134,59375.376887,2,1,251.135458,-23.703780,18.983727,0.160271
7,ZTF21abgavxk,1624308764315015004,0.0,19.3,2003FP134,59378.308762,2,1,250.466414,-23.403652,19.122366,0.093317
8,ZTF21abgxhjg,1628315454015015046,0.0,19.5,2003FP134,59382.315451,2,1,249.608787,-22.995193,19.243404,0.105018
9,ZTF21abhfyvv,1630314794115015013,0.0,19.5,2003FP134,59384.314792,2,1,249.212822,-22.794441,19.301159,0.215448


In [9]:
query =  '''-- Corrected query to retrieve detections for Solar System identifiers '2003FP134' and '2009UK56'
-- The error was due to the use of a non-existent function `q3c_radial_query` and missing RA/Dec values.
-- This query assumes that the RA/Dec values for the Solar System identifiers are not available in the schema,
-- so we will filter based on the `ssdistnr` column (distance to the nearest known Solar System object).

SELECT ss.oid,
       ss.candid,
       ss.ssdistnr,
       ss.ssmagnr,
       ss.ssnamenr,
       d.mjd AS detection_date,
       d.fid AS filter_identifier,
       d.isdiffpos,
       d.ra,
       d.dec,
       d.magpsf AS difference_magnitude,
       d.sigmapsf AS difference_magnitude_uncertainty
FROM ss_ztf ss
JOIN detection d ON ss.candid = d.candid -- Join on candid as it uniquely identifies detections

WHERE ss.ssnamenr IN ('2003FP134',
                      '2009UK56')-- Filter for the specified Solar System identifiers

  AND ss.ssdistnr <= 2.0 / 3600.0 -- Filter for objects within 2 arcseconds (converted to degrees)
ORDER BY ss.ssnamenr, -- Sort by Solar System identifier
 d.mjd'''
n = 3
tables_i, error = run_sql_alerce( query, access_time=2, n_tries=n, query_time=False )
if error:
    print(f'Error in query: {error}')
    # query_columns.append({"req_id": row.req_id, "cols": [], "shape": []})
else:
    print(f'Query executed successfully, shape: {tables_i.shape}')
    # query_columns.append({"req_id": row.req_id, "cols": tables_i.columns.values.tolist(), "shape": tables_i.shape})
tables_i

Query executed successfully, shape: (27, 12)


,oid,candid,ssdistnr,ssmagnr,ssnamenr,detection_date,filter_identifier,isdiffpos,ra,dec,difference_magnitude,difference_magnitude_uncertainty
0,ZTF21abbsmyp,1595376262715015044,0.0,19.5,2003FP134,59349.376262,1,1,256.824875,-25.974222,20.192154,0.178280
1,ZTF21abcqekm,1608313551615015025,0.0,19.0,2003FP134,59362.313553,2,1,254.248201,-24.971674,18.976067,0.170550
2,ZTF21abculin,1609376562915015021,0.0,19.0,2003FP134,59363.376562,2,1,254.000931,-24.876152,19.098774,0.178708
3,ZTF21abdihfx,1612381041715015014,0.0,18.9,2003FP134,59366.381042,2,1,253.289819,-24.597101,19.052930,0.091880
4,ZTF21abdulcm,1614399211715015002,0.0,18.8,2003FP134,59368.399213,2,1,252.804054,-24.403010,18.866552,0.111349
5,ZTF21abeywpa,1619365033915015076,0.0,19.1,2003FP134,59373.365035,2,1,251.609834,-23.908624,19.000805,0.124175
6,ZTF21abfkygj,1621376883815015021,0.0,19.2,2003FP134,59375.376887,2,1,251.135458,-23.703780,18.983727,0.160271
7,ZTF21abgavxk,1624308764315015004,0.0,19.3,2003FP134,59378.308762,2,1,250.466414,-23.403652,19.122366,0.093317
8,ZTF21abgxhjg,1628315454015015046,0.0,19.5,2003FP134,59382.315451,2,1,249.608787,-22.995193,19.243404,0.105018
9,ZTF21abhfyvv,1630314794115015013,0.0,19.5,2003FP134,59384.314792,2,1,249.212822,-22.794441,19.301159,0.215448


In [ ]:
# query 96
query = '''-- Final Query to retrieve all detections for ZTF objects within 2 arcseconds of Solar System identifiers '2003FP134' and '2009UK56'
WITH solar_system_objects AS ( -- Define the RA/Dec coordinates for the Solar System identifiers 
SELECT DISTINCT ssnamenr, oid, candid, ssdistnr, ssmagnr FROM ss_ztf WHERE ssnamenr IN ('2003FP134', '2009UK56')),
filtered_objects AS ( -- Perform spatial filtering using q3c_radial_query to find objects within 2 arcseconds 
SELECT sso.*, d.* FROM solar_system_objects sso INNER JOIN detection d ON sso.candid = d.candid 
WHERE q3c_radial_query(d.ra, d.dec, sso.ssdistnr, 0.0005556) -- 2 arcseconds in degrees
) -- Select required columns and sort the results
SELECT  sso.*,  d.mjd AS detection_date,  d.fid AS filter_identifier,  d.isdiffpos,  d.ra,  d.dec,  d.magpsf AS difference_magnitude,  d.sigmapsf AS magnitude_uncertainty 
FROM filtered_objects sso
ORDER BY sso.ssnamenr, d.mjd'''

query = '''-- Final Query to retrieve all detections for ZTF objects within 2 arcseconds of Solar System identifiers '2003FP134' and '2009UK56'
WITH catalog AS (    -- Define the Solar System objects of interest with their RA/Dec coordinates    
SELECT '2003FP134' AS source_id, ra_value1 AS ra, dec_value1 AS dec
UNION ALL    SELECT '2009UK56' AS source_id, ra_value2 AS ra, dec_value2 AS dec)
SELECT     ss_ztf.*, -- All columns from the ss_ztf table    
detection.mjd AS detection_date, -- Detection date
detection.fid AS filter_identifier, -- Filter identifier
detection.isdiffpos, -- Isdiffpos flag
detection.ra, -- RA coordinate
detection.dec, -- Dec coordinate
detection.magpsf AS difference_magnitude, -- Difference magnitude    
detection.sigmapsf AS magnitude_uncertainty -- Magnitude uncertainty
FROM     ss_ztf
INNER JOIN     catalog    ON q3c_radial_query(ss_ztf.ra, ss_ztf.dec, catalog.ra, catalog.dec, 0.0005556) -- Spatial filtering within 2 arcseconds
INNER JOIN     detection    ON ss_ztf.oid = detection.oid -- Join with detection table on oid
WHERE     ss_ztf.ssnamenr IN ('2003FP134', '2009UK56') -- Filter for specific Solar System objects
ORDER BY     ss_ztf.ssnamenr ASC, -- Sort by Solar System object name    
detection.mjd ASC -- Sort by detection date'''

query = '''-- [FINAL QUERY HERE]
SELECT ss_ztf.*, detection.mjd AS detection_date, detection.fid AS filter_identifier, detection.isdiffpos, detection.ra, detection.dec, 
detection.magpsf AS difference_magnitude, detection.sigmapsf AS magnitude_uncertainty
FROM ss_ztf
INNER JOIN detection ON ss_ztf.oid = detection.oid
WHERE ss_ztf.ssnamenr IN ('2003FP134', '2009UK56')
AND q3c_radial_query(detection.ra, detection.dec, ss_ztf.ra, ss_ztf.dec, 0.0005556)
ORDER BY ss_ztf.ssnamenr ASC, detection.mjd ASC'''


n = 3
tables_i, error = run_sql_alerce( query, access_time=2, n_tries=n, query_time=False )
if error:
    print(f'Error in query: {error}')
    # query_columns.append({"req_id": row.req_id, "cols": [], "shape": []})
else:
    print(f'Query executed successfully, shape: {tables_i.shape}')
    # query_columns.append({"req_id": row.req_id, "cols": tables_i.columns.values.tolist(), "shape": tables_i.shape})
tables_i

Query executed successfully, shape: (1, 7)


,oid,candid,neargaia,neargaiabright,maggaia,maggaiabright,unique1
0,ZTF18acxlskz,1917201552615010028,0.74125,-999.0,15.321053,-999.0,False


In [8]:
db_test[db_test.req_id == 90].request.values[0]

"Find at most 100 ZTF objects that have a multiband period lower than 5 days in the 'lc_classifier_1.2.1-P' feature version. Return all columns from the 'probability' table for such objects, including only data for the light curve classifier, with rankings either 1 or 2"

In [12]:
print(db_test[db_test.req_id == 90].gold_query.values[0])

SELECT
    *
FROM
    probability
WHERE
    classifier_name = 'lc_classifier'
    AND ranking <= 2
    AND oid in
(
SELECT
    oid
FROM
    feature
WHERE
    name = 'Multiband_period'
    AND version = 'lc_classifier_1.2.1-P'
    AND value < 5
LIMIT 100
)


In [36]:
db_test[db_test.req_id == 90].request.values[0]


"Find at most 100 ZTF objects that have a multiband period lower than 5 days in the 'lc_classifier_1.2.1-P' feature version. Return all columns from the 'probability' table for such objects, including only data for the light curve classifier, with rankings either 1 or 2"

In [30]:
# query 90
query = '''SELECT DISTINCT ON (oid)
    *
FROM
    probability
WHERE
    classifier_name = 'lc_classifier'
    AND ranking <= 2
    AND oid in
(
SELECT
    oid
FROM
    feature
WHERE
    name = 'Multiband_period'
    AND version = 'lc_classifier_1.2.1-P'
    AND value < 5
LIMIT 100
)'''
# # run 1 simon 
# # mutiband_period with lower case
# # limit 100 at the end, not inside the subquery
# query='''
# -- Final Query to retrieve at most 100 ZTF objects with a multiband period lower than 5 days
# -- in the 'lc_classifier_1.2.1-P' feature version, returning all columns from the 'probability' table
# -- for the light curve classifier with rankings 1 or 2.
# WITH filtered_features AS (    -- Subquery 1: Filter objects based on multiband period
#     SELECT oid    FROM feature    WHERE name = 'multiband_period'      AND value < 5
#     AND version = 'lc_classifier_1.2.1-P'),
#     filtered_probabilities AS (    -- Subquery 2: Filter probabilities for the light curve classifier
#     SELECT *    FROM probability    WHERE classifier_name = 'lc_classifier'
#     AND ranking IN (1, 2)) -- Combine the filters using an INNER JOIN
#     SELECT p.*
#     FROM filtered_probabilities p
#     INNER JOIN filtered_features f
#     ON p.oid = f.oid-- Limit the results to at most 100 objects
#     LIMIT 100
# '''
# # run 2 simon
# # mutiband_period with upper case
# # limit 100 at the end, not inside the subquery
# query='''
# -- Final Query to retrieve at most 100 ZTF objects with a multiband period lower than 5 days
# -- in the 'lc_classifier_1.2.1-P' feature version, returning all columns from the 'probability' table
# -- for the light curve classifier with rankings 1 or 2.
# WITH filtered_features AS (    -- Subquery 1: Filter objects based on multiband period
#     SELECT oid    FROM feature
#         WHERE name = 'Multiband_period'
#               AND value < 5
#               AND version = 'lc_classifier_1.2.1-P'),
#               filtered_probabilities AS (    -- Subquery 2: Filter probabilities for the light curve classifier
#                 SELECT *    FROM probability    
#                 WHERE classifier_name = 'lc_classifier'      
#                 AND ranking IN (1, 2))-- Combine the filters using an INNER JOIN
#                 SELECT p.*
#                 FROM filtered_probabilities p
#                 INNER JOIN filtered_features f
#                 ON p.oid = f.oid -- Limit the results to at most 100 objects
#                 LIMIT 100
# '''
# # run 3 simon
# query='''
# -- Final Query to retrieve at most 100 ZTF objects with a multiband period lower than 5 days
# -- in the 'lc_classifier_1.2.1-P' feature version, returning all columns from the 'probability' table
# -- for the light curve classifier with rankings of 1 or 2.
# WITH filtered_features AS (    -- Subquery 1: Filter the 'feature' table for objects with multiband_period < 5
#     SELECT oid    FROM feature    
#     WHERE name = 'Multiband_period'      
#     AND value < 5      
#     AND version = 'lc_classifier_1.2.1-P'),
#     filtered_probabilities AS (    -- Subquery 2: Filter the 'probability' table for the light curve classifier with rankings 1 or 2
#         SELECT *    FROM probability    
#         WHERE classifier_name = 'lc_classifier'      
#         AND (ranking = 1 OR ranking = 2))-- Final Query: Join the filtered results and limit to 100 rows
#         SELECT p.*
#         FROM filtered_probabilities p
#         INNER JOIN filtered_features f
#         ON p.oid = f.oid
#         LIMIT 100
# '''
n = 3
tables_i, error = run_sql_alerce( query, access_time=2, n_tries=n, query_time=False )
if error:
    print(f'Error in query: {error}')
    # query_columns.append({"req_id": row.req_id, "cols": [], "shape": []})
else:
    print(f'Query executed successfully, shape: {tables_i.shape}')
    # query_columns.append({"req_id": row.req_id, "cols": tables_i.columns.values.tolist(), "shape": tables_i.shape})
tables_i

Query executed successfully, shape: (100, 6)


,oid,classifier_name,classifier_version,class_name,probability,ranking
0,ZTF17aaaessu,lc_classifier,lc_classifier_1.1.13,CEP,0.225984,2
1,ZTF17aaagqxl,lc_classifier,hierarchical_rf_1.1.0,E,0.317112,1
2,ZTF17aabwftj,lc_classifier,lc_classifier_1.1.13,Periodic-Other,0.262268,1
3,ZTF17aacpvbf,lc_classifier,hierarchical_rf_1.1.0,E,0.494016,1
4,ZTF18aaydigk,lc_classifier,hierarchical_rf_1.1.0,YSO,0.494768,1
...,...,...,...,...,...,...
95,ZTF21aaaqswa,lc_classifier,hierarchical_rf_1.1.0,AGN,0.132756,2
96,ZTF21aaguuxw,lc_classifier,hierarchical_rf_1.1.0,Periodic-Other,0.280500,1
97,ZTF21abcdsuv,lc_classifier,hierarchical_rf_1.1.0,E,0.426688,1
98,ZTF22aabwnvh,lc_classifier,hierarchical_rf_1.1.0,QSO,0.678160,1


In [19]:
# query 90
query = '''SELECT 
    *
FROM
    probability
WHERE
    classifier_name = 'lc_classifier'
    AND ranking <= 2
    AND oid in
(
SELECT
    oid
FROM
    feature
WHERE
    name = 'Multiband_period'
    AND version = 'lc_classifier_1.2.1-P'
    AND value < 5
LIMIT 100
)'''
n = 3
tables_i, error = run_sql_alerce( query, access_time=2, n_tries=n, query_time=False )
if error:
    print(f'Error in query: {error}')
    # query_columns.append({"req_id": row.req_id, "cols": [], "shape": []})
else:
    print(f'Query executed successfully, shape: {tables_i.shape}')
    # query_columns.append({"req_id": row.req_id, "cols": tables_i.columns.values.tolist(), "shape": tables_i.shape})
tables_i

Query executed successfully, shape: (346, 6)


,oid,classifier_name,classifier_version,class_name,probability,ranking
0,ZTF19abdyprc,lc_classifier,lc_classifier_1.1.13,CEP,0.161280,2
1,ZTF19abdyprc,lc_classifier,hierarchical_rf_1.1.0,DSCT,0.283360,1
2,ZTF19abdyprc,lc_classifier,lc_classifier_1.1.13,RRL,0.190720,1
3,ZTF19abdyprc,lc_classifier,hierarchical_rf_1.1.0,RRL,0.227040,2
4,ZTF18abadjvy,lc_classifier,hierarchical_rf_1.1.0,E,0.093744,2
...,...,...,...,...,...,...
341,ZTF18abasovn,lc_classifier,hierarchical_rf_1.1.0,E,0.424664,1
342,ZTF18abasovn,lc_classifier,lc_classifier_1.1.13,E,0.309624,1
343,ZTF18abasovn,lc_classifier,hierarchical_rf_1.1.0,RRL,0.188956,2
344,ZTF22aabwnvh,lc_classifier,hierarchical_rf_1.1.0,Blazar,0.205800,2


In [29]:
sorted(tables_i.oid.unique())

['ZTF17aaaessu',
 'ZTF17aaagqxl',
 'ZTF17aabwftj',
 'ZTF17aacpvbf',
 'ZTF18aaydigk',
 'ZTF18aayhppd',
 'ZTF18aaynqnn',
 'ZTF18aaynqox',
 'ZTF18aaypmtr',
 'ZTF18aayujpj',
 'ZTF18aayutyl',
 'ZTF18aayuugb',
 'ZTF18aayvacc',
 'ZTF18aayvbrz',
 'ZTF18aazwigp',
 'ZTF18abadjvy',
 'ZTF18abaqmri',
 'ZTF18abasovn',
 'ZTF18abcxguj',
 'ZTF18abgsptz',
 'ZTF18abilxvt',
 'ZTF18abivpiy',
 'ZTF18abkjhnt',
 'ZTF18abkmmts',
 'ZTF18ablpmbs',
 'ZTF18ablqndm',
 'ZTF18ablvyyw',
 'ZTF18abmjosl',
 'ZTF18abmrcsn',
 'ZTF18abommtg',
 'ZTF18aboskew',
 'ZTF18abpogdx',
 'ZTF18abrnfpd',
 'ZTF18abrrinj',
 'ZTF18abscvmd',
 'ZTF18absitkp',
 'ZTF18absoocn',
 'ZTF18abspqgp',
 'ZTF18abvgrec',
 'ZTF18abvztqd',
 'ZTF18acajzhc',
 'ZTF18acavigy',
 'ZTF18acdasee',
 'ZTF18acfrjme',
 'ZTF18achfgss',
 'ZTF18achixag',
 'ZTF18aciepmk',
 'ZTF18acnambs',
 'ZTF18acrugci',
 'ZTF18acsbrqc',
 'ZTF18actytgu',
 'ZTF18acuwvpr',
 'ZTF18acwbhzi',
 'ZTF18adamiis',
 'ZTF18adamzmn',
 'ZTF18adaoqcm',
 'ZTF18adapbif',
 'ZTF18adapnnv',
 'ZTF18adaqdfq

In [31]:
sorted(tables_i.oid.unique())

['ZTF17aaaessu',
 'ZTF17aaagqxl',
 'ZTF17aabwftj',
 'ZTF17aacpvbf',
 'ZTF18aaydigk',
 'ZTF18aayhppd',
 'ZTF18aaynqnn',
 'ZTF18aaynqox',
 'ZTF18aaypmtr',
 'ZTF18aayujpj',
 'ZTF18aayutyl',
 'ZTF18aayuugb',
 'ZTF18aayvacc',
 'ZTF18aayvbrz',
 'ZTF18aazwigp',
 'ZTF18abadjvy',
 'ZTF18abaqmri',
 'ZTF18abasovn',
 'ZTF18abcxguj',
 'ZTF18abgsptz',
 'ZTF18abilxvt',
 'ZTF18abivpiy',
 'ZTF18abkjhnt',
 'ZTF18abkmmts',
 'ZTF18ablpmbs',
 'ZTF18ablqndm',
 'ZTF18ablvyyw',
 'ZTF18abmjosl',
 'ZTF18abmrcsn',
 'ZTF18abommtg',
 'ZTF18aboskew',
 'ZTF18abpogdx',
 'ZTF18abrnfpd',
 'ZTF18abrrinj',
 'ZTF18abscvmd',
 'ZTF18absitkp',
 'ZTF18absoocn',
 'ZTF18abspqgp',
 'ZTF18abvgrec',
 'ZTF18abvztqd',
 'ZTF18acajzhc',
 'ZTF18acavigy',
 'ZTF18acdasee',
 'ZTF18acfrjme',
 'ZTF18achfgss',
 'ZTF18achixag',
 'ZTF18aciepmk',
 'ZTF18acnambs',
 'ZTF18acrugci',
 'ZTF18acsbrqc',
 'ZTF18actytgu',
 'ZTF18acuwvpr',
 'ZTF18acwbhzi',
 'ZTF18adamiis',
 'ZTF18adamzmn',
 'ZTF18adaoqcm',
 'ZTF18adapbif',
 'ZTF18adapnnv',
 'ZTF18adaqdfq

In [39]:
# query 84
query = '''
SELECT
    oid, xmatch.oid_catalog, dist,
    w1mpro, w2mpro, w3mpro, w4mpro
FROM
    xmatch INNER JOIN
    allwise
    ON xmatch.oid_catalog = allwise.oid_catalog
WHERE
    xmatch.oid = 'ZTF19aascdol'
'''
# # run 2 simon 
query = '''
-- Retrieve information about the ZTF object 'ZTF19aascdol' and its ALLWISE match(es)
SELECT     object.oid AS ztf_identifier, -- ZTF object identifier    
allwise.oid_catalog AS allwise_identifier, -- ALLWISE catalog identifier    
q3c_dist(object.meanra, object.meandec, allwise.ra, allwise.dec) AS distance, -- Angular distance between counterparts    
allwise.w1mpro, -- WISE W1 magnitude    
allwise.w2mpro, -- WISE W2 magnitude    
allwise.w3mpro, -- WISE W3 magnitude    
allwise.w4mpro  -- WISE W4 magnitude
FROM     object
INNER JOIN     allwise
ON     q3c_join(object.meanra, object.meandec, allwise.ra, allwise.dec, 0.00138889) -- Match within 5 arcseconds (0.00138889 degrees)
WHERE     object.oid = 'ZTF19aascdol' -- Filter for the specific ZTF object
ORDER BY     distance ASC -- Sort by angular distance to prioritize the closest match
'''
n = 3
tables_i, error = run_sql_alerce( query, access_time=2, n_tries=n, query_time=False )
if error:
    print(f'Error in query: {error}')
    # query_columns.append({"req_id": row.req_id, "cols": [], "shape": []})
else:
    print(f'Query executed successfully, shape: {tables_i.shape}')
    # query_columns.append({"req_id": row.req_id, "cols": tables_i.columns.values.tolist(), "shape": tables_i.shape})
tables_i

Query executed successfully, shape: (1, 7)


,ztf_identifier,allwise_identifier,distance,w1mpro,w2mpro,w3mpro,w4mpro
0,ZTF19aascdol,J154334.93+152539.5,0.000038,15.286,14.354,11.747,8.611
